In [1]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import pymongo

In [2]:

# Initialize PyMongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.items

# Headlines

In [4]:
#url and get page with requests
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [5]:
#get your soup (object)
headlines_soup = bs(response.text, 'lxml')

In [6]:
#scrape the latest news headline text and get rid of the white space
headline = headlines_soup.find('div', class_ = 'content_title').text.strip()

In [7]:
#print it to make sure it works
print(headline)

Dust Storms Linked to Gas Escape from Mars Atmosphere


In [8]:
#scrape the blurb text that goes with the headline - a.k.a. the first paragraph without white space 
news = headlines_soup.find('div', class_ = 'rollover_description_inner').text.strip()

In [9]:
#make sure that one worked too...
print(news)

If Mars has a global dust storm in 2018, observations could aid understanding of its effects.


# Get your featured image of Mars

In [10]:
# Import Selenium & Splinter and set the chromedriver path (I'm on windows)
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [11]:
#visit the URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [12]:
#click the button to get to the full image
browser.click_link_by_id('full_image')

In [13]:
#pause 2 seconds to let it load
time.sleep(2)

In [14]:
#scrape browser into soup and find full resolution image of Mars
#btw, there is no "largesize" image link
#Bring up the image
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [15]:
#append the url snippet to make it the full url of the image 
featured_img_url = "https://www.jpl.nasa.gov" + img_url

In [16]:
#verify we have the url
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19968_ip.jpg'

# Mars Weather Tweet

In [17]:
#on to getting the latest tweet about the weather on Mars
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [18]:
# create soup object
weather_soup = bs(response.text, 'html.parser')

In [19]:
#grab all the tweets in case the first one isn't the one you want
mars_weather_tweet = weather_soup.find_all('div', class_ = "js-tweet-text-container")

In [20]:
# find the first actual weather tweet (they start with "sol")
for tweet in mars_weather_tweet:
    if tweet.text.strip().startswith('Sol'):
        mars_weather = tweet.text.strip()

In [21]:
#verify you got the right info 
print(mars_weather)

Sol 1910 (Dec 20, 2017), Sunny, high -23C/-9F, low -80C/-112F, pressure at 7.94 hPa, daylight 05:49-17:32


# facts

In [22]:
#visit the space facts site
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [23]:
# find the html table and convert to pandas dataframe
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [24]:
#set column headers 
mars_df.columns = ["Description", "Value"]
# mars_df = mars_df.set_index("Description")
# I decided not to reset the index because, to me, it looks messier to have the column headers in two mis-aligned cells

In [25]:
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [26]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

'<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>3</th>       <td>Moons:</td>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>4</th>       <td>Orbit Distance:</td>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>5</th>       <td>Orbit Period:</td>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>6</th>       <td>Surface Temperature:</td>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>7</th>       <td>First Record:</td>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>8</t

# database

In [32]:
#dictionary to be inserted as a MongoDB document
post = {
    'headline': headline, 
    'news': news, 
    'featured': featured_img_url,
    'weather': mars_weather,
    'facts': table_data
}
collection.insert_one(post)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

In [31]:
webdatas = db.items.find()

for webdata in webdatas:
    print(webdata)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it